## Load Marigold Pipeline

In [4]:
import diffusers
import torch
from diffusers.models.attention_processor import AttnProcessor2_0


device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = diffusers.MarigoldDepthPipeline.from_pretrained(
    "prs-eth/marigold-depth-v1-1", variant="fp16", torch_dtype=torch.float16 # commented this to make it run smoothly on Apple Silicon M1+
).to(device)

# pipe.vae = torch.compile(pipe.vae, mode="reduce-overhead", fullgraph=True)
# pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Compile to speed up the pipeline and reduce memory usage.


In [5]:
import os
from PIL import Image

input_folder = "image-net/ILSVRC2012_img_val"
output_folder = "image-net-depth/ILSVRC2012_img_val"
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.JPEG', '.JPG'))]

image_files.sort()

start_from = 49999
end_at = 50001

print(len(image_files[start_from:end_at]), "images to process")

1 images to process


In [6]:
from tqdm.auto import tqdm

pipe.set_progress_bar_config(disable=True)

for n in tqdm(range(start_from, end_at), desc="Processing images"):
    image_path = os.path.join(input_folder, image_files[n])
    image = Image.open(image_path)
    # Ensure the image is in RGB mode
    if image.mode != 'RGB':
        image = image.convert('RGB')
    depth = pipe(image)
    visualized_depth = pipe.image_processor.export_depth_to_16bit_png(depth.prediction[0])
    visualized_depth[0].save(os.path.join(output_folder, f"ILSVRC2012_val_{n:08d}.png"))
    

Processing images:   0%|          | 0/2 [00:00<?, ?it/s]

IndexError: list index out of range